# Tutorial: Forte2's determinant class

---

In this tutorial we are going to explore how to create a simple FCI code using forte's Python API.

In [1]:
from forte2 import Determinant

## Creating determinants and interacting with them

Determinants are represented by the class `Determinant`. Here we create a determinant with all occupation numbers set to zero via the class function `zero()`.
When we print the determinant, we see the occupation of each orbital:

In [2]:
d = Determinant.zero()
print(f'Determinant: {d}')

Determinant: |0000000000000000000000000000000000000000000000000000000000000000>


If we are working with only a few orbitals we can use the `.str()` function to produce a more compact representation:

In [3]:
print(f'Determinant: {d.str(4)}') # print only the occupation of 4 orbitals

Determinant: |0000>


There are several ways one can create determinants. The utility function `forte.det` converts a string to a determinant:

In [4]:
d = Determinant("2a0bab")
d

|2a0bab0000000000000000000000000000000000000000000000000000000000>

We can check the occupation of a specific orbital using the `na` and `nb` methods:

In [5]:
d = Determinant("2a0bab")
# Access occupation numbers of first two orbitals
d.na(0), d.nb(0), d.na(1), d.nb(1)

(True, True, True, False)

The occupation of a spin orbital can be set with the `set_na` and `set_nb` member functions, passing the orbital index and the value (`True` = 1, `False` = 0).

In [6]:
d = Determinant.zero()
# Set occupation of orbital 2 alpha and orbital 1 beta to 1
d.set_na(2, True)
d.set_nb(1, True)
d

|0ba0000000000000000000000000000000000000000000000000000000000000>

Another common operation is counting the number of occupied spinorbitals in a determinant. The functions `count_a`, `count_b`, and `count` return the number of occupied alpha, beta, and alpha + beta orbitals:

In [7]:
d = Determinant("2aba")
d.count_a(), d.count_b(), d.count()

(3, 2, 5)

We can also create a new determinant in which all spins are flipped:

In [8]:
d = Determinant("aabb20")
print(f'Determinant: {d.str(6)}')
new_d = d.spin_flip()
print(f'Determinant: {new_d.str(6)}')

Determinant: |aabb20>
Determinant: |bbaa20>


## Applying creation and annihilation operators to determinants

Determinants can be modified by applying second quantized creation and annihilation operators.
To apply the creation operator $\hat{a}^\dagger_{1 \alpha}$, which adds one electron in the spin orbital $\psi_{1\alpha}$, we can use the function (`create_alfa`). This function returns the corresponding sign and modifies the original determinant (this is done for performance reasons):

In [9]:
d = Determinant.zero()
print(f'Original determinant: {d.str(4)}')

sign = d.create_a(1)
print(f'New determinant:      {d.str(4)}, sign = {sign}')

Original determinant: |0000>
New determinant:      |0a00>, sign = 1.0


Here we create an electron in spin orbital 2 beta 

In [10]:
sign = d.create_b(2)
print(f'Determinant: {d.str(4)}, sign = {sign}')

Determinant: |0ab0>, sign = -1.0


Similarly, we can remove (annihilate) an electron with the command `destroy_alfa` (`destroy_beta` for the beta case). Since this orbital is empty, the sign returned is 0

In [11]:
sign = d.destroy_a(2)
print(f'Determinant: {d.str(4)}, sign = {sign}')

Determinant: |0ab0>, sign = 0.0


If we are only interested in the sign obtained when applying a creation/annihilation operator (irrespective of the occupation), we can use the `slater_sign` function

In [12]:
d = Determinant("a0a")
# Calculate the Slater sign for applying creation/annihilation operators that act
# on the first five orbitals
d.slater_sign(0), d.slater_sign(1), d.slater_sign(2), d.slater_sign(3), d.slater_sign(4)

(1.0, -1.0, -1.0, 1.0, 1.0)

## Comparing determinants

Determinants support comparison (`==`, `!=`, `>`, `<`, etc.) of operators, introducing a canonical ordering of determinants

In [13]:
d1 = Determinant("a")
d2 = Determinant("b")
d3 = Determinant("0a")
d4 = Determinant("a")

# Check if determinants are equal
print(f'{d1.str(2)} == {d2.str(2)}: {d1 == d2}')
print(f'{d1.str(2)} == {d4.str(2)}: {d1 == d4}')

# Check if determinants are not equal
print(f'{d1.str(2)} != {d2.str(2)}: {d1 != d2}')
print(f'{d1.str(2)} != {d3.str(2)}: {d1 != d3}')
print(f'{d1.str(2)} != {d4.str(2)}: {d1 != d4}')

# Check if determinants are less than or greater than
print(f'{d1.str(2)} < {d2.str(2)}: {d1 < d2}')
print(f'{d1.str(2)} < {d3.str(2)}: {d1 < d3}')
print(f'{d1.str(2)} < {d4.str(2)}: {d1 < d4}')

|a0> == |b0>: False
|a0> == |a0>: True
|a0> != |b0>: True
|a0> != |0a>: True
|a0> != |a0>: False
|a0> < |b0>: True
|a0> < |0a>: True
|a0> < |a0>: False


Comparison of `Determinant` enables sorting and finding elements in lists:

In [20]:
# create a list of determinants
dets = [Determinant("a0"),
         Determinant("ab"),
         Determinant("ba"),
         Determinant("20")]

# print a sorted list of determinants
print(sorted(dets))

# Check if determinant |ab> is in the list
if Determinant("ab") in dets:
    print("Found 'ab' in the list of determinants")

[|a000000000000000000000000000000000000000000000000000000000000000>, |2000000000000000000000000000000000000000000000000000000000000000>, |ba00000000000000000000000000000000000000000000000000000000000000>, |ab00000000000000000000000000000000000000000000000000000000000000>]
Found 'ab' in the list of determinants


`Determinant` objects are also hashable, so that they can be used as keys in dictionaries:

In [15]:
# create a dictionary with determinants as keys
id = {}
id[d1] = 0; id[d2] = 1; id[d3] = 2

print(f'Index of {d2.str(2)}: {id[d2]}')

Index of |b0>: 1


## Differences between determinants

We can easily compare two determinants to check differences in occupation. The function `excitation_connection` returns a list of four lists, each containing the orbitals removed and added for each spin case in the order: `alpha removed`, `alpha added`, `beta removed`, `beta added`. In this example, the output shows that d1 is connected to d2 by removing orbital 0 of spin alpha:

In [16]:
d1 = Determinant("20")
d2 = Determinant("b0")

d1.excitation_connection(d2)

[[0], [], [], []]

In the next example, the two determinants differ by a double excitation:

In [17]:
d1 = Determinant("200")
d2 = Determinant("0ab")

d1.excitation_connection(d2)

[[0], [1], [0], [2]]

## Creating a determinant basis

Forte2 provides the utility function `hilbert_space` to create all possible determinants with a given number of electrons in a given number of orbitals. This function runs in C++ and will return a list of determinants.

In [18]:
# Create the Hilbert space of determinants for a space with
# one alpha electron and one beta electron in two orbitals
nmo = 2 # number of orbitals
na = 1  # number of alpha electrons
nb = 1  # number of beta electrons

dets = forte2.hilbert_space(nmo=nmo, na=na, nb=nb)

print(f'Hilbert space determinants:')
for d in dets:
    print(f'{d.str(nmo)}')

NameError: name 'forte2' is not defined

This function can also be passed symmetry information to generate only those determinants that have the desired symmetry and given number of alpha/beta electrons.

Here is an example of how `hilbert_space` can be used to generate the full Fock space of determinants for a given number of orbitals:

In [ ]:
# Create the Fock space of determinants in a two orbital basis
# of dimension 4^2 = 16
nmo = 2 # number of orbitals

dets = []
for na in range(nmo + 1):  # number of alpha electrons
    for nb in range(nmo + 1):  # number of beta electrons
        dets += forte2.hilbert_space(nmo=nmo, na=na, nb=nb)

print(f'Fock space determinants:')
for d in dets:
    print(f'{d.str(nmo)}')

Fock space determinants:
|00>
|0b>
|b0>
|bb>
|0a>
|a0>
|02>
|ba>
|ab>
|20>
|b2>
|2b>
|aa>
|a2>
|2a>
|22>
